In [5]:
%cd /content
!rm -rf KoreanStandardPronunciation
!git clone https://github.com/dhkang01/KoreanStandardPronunciation.git
%cd KoreanStandardPronunciation


/content
Cloning into 'KoreanStandardPronunciation'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 28 (delta 11), reused 18 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 77.69 KiB | 5.55 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/KoreanStandardPronunciation


In [6]:
!pip install -q "transformers>=4.38.0" "datasets>=2.18.0" "peft>=0.11.0" accelerate huggingface_hub evaluate

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
from datasets import load_dataset

dataset_id = "dhkang01/KMA_dataset"
raw_ds = load_dataset(dataset_id, split="train")

raw_ds

README.md:   0%|          | 0.00/356 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/447115 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'input', 'output'],
    num_rows: 447115
})

In [9]:
ds_train_tmp = raw_ds.train_test_split(test_size=0.1, seed=42)
train_ds = ds_train_tmp["train"]
tmp_ds   = ds_train_tmp["test"]

ds_val_test = tmp_ds.train_test_split(test_size=0.5, seed=42)
val_ds = ds_val_test["train"]
test_ds = ds_val_test["test"]

# train/val/test -> 90/5/5

train_ds = train_ds.select(range(10000))
val_ds = val_ds.select(range(100))


Tokenizer 다운로드

In [10]:
from KoCharELECTRA.tokenization_kocharelectra import KoCharElectraTokenizer

model_name = "monologg/kocharelectra-small-discriminator"

tokenizer = KoCharElectraTokenizer.from_pretrained(model_name)
print(tokenizer.tokenize("가나다"))

tokenizer_config.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'KoCharElectraTokenizer'.


['가', '나', '다']


output vocab

In [11]:
from collections import OrderedDict

# tokenizer.vocab은 OrderedDict(토큰 → ID)
token_list = list(tokenizer.vocab.keys())

pron2id = OrderedDict()
for idx, tok in enumerate(token_list):
    pron2id[tok] = idx

id2pron = {v: k for k, v in pron2id.items()}

len(pron2id), list(list(pron2id.items())[:10])

(11568,
 [('[PAD]', 0),
  ('[UNK]', 1),
  ('[CLS]', 2),
  ('[SEP]', 3),
  ('[MASK]', 4),
  (' ', 5),
  ('이', 6),
  ('다', 7),
  ('는', 8),
  ('에', 9)])

전처리 함수 정의 및 적용

복수 발음 허용 X

In [12]:
import numpy as np

max_length = 128  # 필요에 따라 조절

def preprocess_example(example):
    text = example["input"]
    pron = example["output"]  # List[List[str]]

    # KoCharElectra는 char 단위 토큰 + [CLS], [SEP]
    encoding = tokenizer(
        text,
        truncation=True,
        max_length=max_length,
        padding="max_length",  # DataCollator 써도 되지만 여기서는 고정 길이로
        return_tensors=None,
    )

    input_ids = encoding["input_ids"]
    attention_mask = encoding["attention_mask"]

    # Electra/KoCharElectra: 대체로 [CLS] + chars + [SEP]
    # => 실제 문자 수 = len(text)
    # => pron 길이와 len(text)가 맞는다고 가정 (안 맞는 샘플은 나중에 필터 가능)
    seq_len = sum(attention_mask)  # 실제 non-pad 길이
    # [CLS] at 0, [SEP] at seq_len-1, chars in 1..seq_len-2

    labels = np.full_like(input_ids, fill_value=-100)  # default ignore_index

    # 문자 수와 pron 길이 안 맞으면 그냥 전부 ignore(-100)로 두고 스킵되게 할 수도 있음
    # 여기선 일단 최소한으로만 체크
    n_chars = seq_len - 2  # CLS, SEP 제외

    if len(pron) != n_chars:
        # 불일치하는 경우: 전부 padding label로 두고, 나중에 이런 샘플 비율 보고 판단
        print(f"Warning: pron len {len(pron)} != n_chars {n_chars} for text: {text}")
        print(f"in the case, pron: {"".join([l[0] for l in pron])}")
        encoding["labels"] = labels.tolist()
        return encoding

    for i in range(len(pron)):
        cand_list = pron[i]
        if not cand_list:
            continue
        if cand_list[0] == ' ':
            continue
        gold_pron = cand_list[0]               # 첫 후보를 gold label로 사용
        label_id = pron2id[gold_pron]
        token_pos = 1 + i                      # 0: [CLS], 1.. : chars
        if token_pos < seq_len - 1:            # 마지막 [SEP] 전까지만
            labels[token_pos] = label_id

    encoding["labels"] = labels.tolist()
    return encoding

In [13]:
train_tokenized = train_ds.map(
    preprocess_example,
    remove_columns=train_ds.column_names,
)

val_tokenized = val_ds.map(
    preprocess_example,
    remove_columns=val_ds.column_names,
)

# too long seq is out.

# train_tokenized[0]


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

in the case, pron: 미구기 세르비아의 인꿘 타나브 리유로 유고에 대하 눤조를 중단하기로 결정한 사시리나 유고 내 공화국까늬 경계선 변경을 반대하는 미구기나 유럽 꾹까드릐 강경한 성명이 대세르비아를 꿈꾸는 세르비아의 밀로세비치 대통녕에 대한 규타늬 표시라고


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

모델 로드, LoRA 적용

encoder에 LoRA적용
classifier에 LoRA적용X, 전부 trainable

In [14]:
from transformers import AutoModelForTokenClassification

num_labels = len(pron2id)

base_model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
)


pytorch_model.bin:   0%|          | 0.00/44.5M [00:00<?, ?B/s]

Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/kocharelectra-small-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["query", "key", "value", "dense"]  # Electra의 attention/FFN 모듈 이름 기준
)

# print(base_model)
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()
print(model)

trainable params: 3,415,344 || all params: 17,445,216 || trainable%: 19.5775
PeftModelForTokenClassification(
  (base_model): LoraModel(
    (model): ElectraForTokenClassification(
      (electra): ElectraModel(
        (embeddings): ElectraEmbeddings(
          (word_embeddings): Embedding(11568, 128, padding_idx=0)
          (position_embeddings): Embedding(512, 128)
          (token_type_embeddings): Embedding(2, 128)
          (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
        (encoder): ElectraEncoder(
          (layer): ModuleList(
            (0-11): 12 x ElectraLayer(
              (attention): ElectraAttention(
                (self): ElectraSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=256, out_features=256, bias=True)
                    (lora_dropo

char_embed 설정

char->vec(24dim)

emb wrapping

In [16]:
# 각 feature index:
# 0: stop, 1: fricative, 2: affricate, 3: nasal, 4: liquid,
# 5: labial, 6: coronal, 7: dorsal, 8: glottal,
# 9: tense, 10: aspirated

CONSONANT_FEATURES = {
    "ㄱ": [1,0,0,0,0,  0,0,1,0, 0,0],  # velar stop
    "ㄲ": [1,0,0,0,0,  0,0,1,0, 1,0],  # tense velar stop
    "ㄴ": [0,0,0,1,0,  0,1,0,0, 0,0],  # coronal nasal
    "ㄷ": [1,0,0,0,0,  0,1,0,0, 0,0],  # coronal stop
    "ㄸ": [1,0,0,0,0,  0,1,0,0, 1,0],  # tense coronal stop
    "ㄹ": [0,0,0,0,1,  0,1,0,0, 0,0],  # coronal liquid
    "ㅁ": [0,0,0,1,0,  1,0,0,0, 0,0],  # labial nasal
    "ㅂ": [1,0,0,0,0,  1,0,0,0, 0,0],  # labial stop
    "ㅃ": [1,0,0,0,0,  1,0,0,0, 1,0],  # tense labial stop
    "ㅅ": [0,1,0,0,0,  0,1,0,0, 0,0],  # coronal fricative
    "ㅆ": [0,1,0,0,0,  0,1,0,0, 1,0],  # tense coronal fricative
    "ㅇ": [0,0,0,1,0,  0,0,1,0, 0,0],  # velar nasal (종성 기준)
    "ㅈ": [0,0,1,0,0,  0,1,0,0, 0,0],  # coronal affricate
    "ㅉ": [0,0,1,0,0,  0,1,0,0, 1,0],  # tense coronal affricate
    "ㅊ": [0,0,1,0,0,  0,1,0,0, 0,1],  # aspirated coronal affricate
    "ㅋ": [1,0,0,0,0,  0,0,1,0, 0,1],  # aspirated velar stop
    "ㅌ": [1,0,0,0,0,  0,1,0,0, 0,1],  # aspirated coronal stop
    "ㅍ": [1,0,0,0,0,  1,0,0,0, 0,1],  # aspirated labial stop
    "ㅎ": [0,1,0,0,0,  0,0,0,1, 0,1],  # glottal fricative, aspirated
}

CODA_COMPOUND = {
    "ㄳ": ("ㄱ", "ㅅ"),
    "ㄵ": ("ㄴ", "ㅈ"),
    "ㄶ": ("ㄴ", "ㅎ"),
    "ㄺ": ("ㄹ", "ㄱ"),
    "ㄻ": ("ㄹ", "ㅁ"),
    "ㄼ": ("ㄹ", "ㅂ"),
    "ㄽ": ("ㄹ", "ㅅ"),
    "ㄾ": ("ㄹ", "ㅌ"),
    "ㄿ": ("ㄹ", "ㅍ"),
    "ㅀ": ("ㄹ", "ㅎ"),
    "ㅄ": ("ㅂ", "ㅅ"),
}


In [17]:
# vowel feature index:
# 0: high, 1: mid, 2: low,
# 3: front, 4: central, 5: back,
# 6: round, 7: tense, 8: diphthong

BASE_VOWEL_FEATURES = {
    "ㅏ": [0,0,1,  0,0,1,  0,1,0],  # low back unround, tense-ish
    "ㅓ": [0,1,0,  0,0,1,  0,0,0],  # mid back unround, lax-ish
    "ㅗ": [0,1,0,  0,0,1,  1,1,0],  # mid back round, tense-ish
    "ㅜ": [0,1,0,  0,0,1,  1,0,0],  # mid back round, lax-ish
    "ㅡ": [0,1,0,  0,1,0,  0,0,0],  # mid central unround
    "ㅣ": [1,0,0,  1,0,0,  0,1,0],  # high front unround, tense

    "ㅐ": [0,1,0,  1,0,0,  0,1,0],  # mid front unround, tense
    "ㅔ": [0,1,0,  1,0,0,  0,0,0],  # mid front unround, lax-ish

    # 필요하면 ㅚ, ㅟ, ㅢ도 base로 직접 정의 가능
    "ㅚ": [0,1,0,  0,0,1,  1,1,0],  # /we/ or /ø/ 계열
    "ㅟ": [1,0,0,  1,0,0,  1,1,0],  # /y/ 계열
    "ㅢ": [0,1,0,  0,1,0,  0,1,0],  # central-ish compound
}

COMPOSED_VOWELS = {
    "ㅑ": ("ㅣ", "ㅏ"),
    "ㅒ": ("ㅣ", "ㅐ"),
    "ㅕ": ("ㅣ", "ㅓ"),
    "ㅖ": ("ㅣ", "ㅔ"),
    "ㅛ": ("ㅣ", "ㅗ"),
    "ㅠ": ("ㅣ", "ㅜ"),

    "ㅘ": ("ㅗ", "ㅏ"),
    "ㅙ": ("ㅗ", "ㅐ"),
    "ㅝ": ("ㅜ", "ㅓ"),
    "ㅞ": ("ㅜ", "ㅔ"),
}

In [18]:
def get_consonant_feature(jamo):
    if jamo in CONSONANT_FEATURES:
        return np.array(CONSONANT_FEATURES[jamo], dtype=np.float32)

    if jamo in CODA_COMPOUND:
        a, b = CODA_COMPOUND[jamo]
        return (get_consonant_feature(a) + get_consonant_feature(b)) / 2

    raise ValueError(f"Unknown consonant: {jamo}")


def get_vowel_feature(jamo):
    if jamo in BASE_VOWEL_FEATURES:
        return np.array(BASE_VOWEL_FEATURES[jamo], dtype=np.float32)

    if jamo in COMPOSED_VOWELS:
        a, b = COMPOSED_VOWELS[jamo]
        return (get_vowel_feature(a) + get_vowel_feature(b)) / 2

    raise ValueError(f"Unknown vowel: {jamo}")

In [19]:
ONSETS  = list("ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ")
NUCLEI  = list("ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ")
CODAS   = [""] + list("ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ")

def decompose(syllable):
    code = ord(syllable) - 0xAC00
    onset_idx = code // 588
    nucleus_idx = (code % 588) // 28
    coda_idx = code % 28

    onset = ONSETS[onset_idx]
    nucleus = NUCLEI[nucleus_idx]
    coda = CODAS[coda_idx] if coda_idx != 0 else None

    return onset, nucleus, coda

In [20]:
def get_syllable_feature(syllable):
    onset, nucleus, coda = decompose(syllable)

    onset_feat  = get_consonant_feature(onset)     # 11D
    nucleus_feat = get_vowel_feature(nucleus)      # 9D

    if coda is None:
        coda_feat = np.zeros(11)                 # 종성 없음 → 11D zero
    else:
        coda_feat = get_consonant_feature(coda)  # 11D

    return np.concatenate([onset_feat, nucleus_feat, coda_feat])  # 31D


char_embed = np.zeros((len(pron2id), 31), dtype=np.float32) # (11568, 31)
print(char_embed.size())

for pron, idx in pron2id.items():
    if len(pron) != 1:
        continue
    if ord(pron) < ord('가'):
        continue
    if ord('힣') < ord(pron):
        continue
    char_embed[idx] = get_syllable_feature(pron)

In [ ]:
class ElectraEmbeddingWithNew(nn.Module):
    def __init__(self, electra_embeddings, char_embed):
        super().__init__()
        self.old = electra_embeddings                  # 기존 ElectraEmbeddings
        self.embed_proj = electra_embeddings.project   # 128 → 256 projection

        # new embedding (11172X31), 학습하지 않음
        self.register_buffer("char_embed", char_embed, persistent=True)

        # new embedding → 256 projection
        self.new_up = nn.Linear(31, 256)

    def forward(self, input_ids, token_type_ids=None):
        # ---- 기존 embedding 128 차원 ----
        old_emb_128 = self.old.word_embeddings(input_ids)
        if token_type_ids is not None:
            tt = self.old.token_type_embeddings(token_type_ids)
            pos = self.old.position_embeddings(torch.arange(input_ids.size(1), device=input_ids.device))
            pos = pos.unsqueeze(0).expand_as(old_emb_128)
            old_emb_128 = old_emb_128 + tt + pos
        old_emb_128 = self.old.LayerNorm(old_emb_128)
        old_emb_128 = self.old.dropout(old_emb_128)

        # ---- Electra projection ----
        old_emb_256 = self.embed_proj(old_emb_128)

        # ---- new embedding 31 차원 ----
        new_emb_31 = self.char_embed[input_ids]     # (B, L, 31)
        new_emb_256 = self.new_up(new_emb_31)

        # ---- 256 차원에서 add ----
        final_emb = old_emb_256 + new_emb_256
        return final_emb


Concat wrapper 모듈 적용

In [ ]:
import torch
import torch.nn as nn

class ElectraConcatEmbeddingClassifier(nn.Module):
    def __init__(self, peft_model, num_labels):
        super().__init__()
        self.peft_model = peft_model  # PeftModelForTokenClassification
        # base_model = ElectraForTokenClassification
        self.base_model = peft_model.base_model
        self.ignore_index = -100

        self.electra = self.base_model.electra   # ElectraModel (LoRA 포함)
        old_emb = self.electra.embeddings
        new_emb = ElectraEmbeddingWithNew(old_emb, char_embed)
        self.char_embed = new_emb.char_embed
        self.electra.embeddings = new_emb
        self.num_labels = num_labels

        self.config = self.base_model.config
        hidden_size = self.config.hidden_size          # 256
        embed_min = char_embed.size(1)

        self.to_31 = nn.Linear(hidden_size, 31)

        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None,
        **kwargs,
    ):
        # 1) input embedding
        embedding_output = self.electra.embeddings(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
        )  # (B, L, embed_dim)

        extended_mask = self.electra.get_extended_attention_mask(
            attention_mask,
            input_shape=input_ids.shape,
            device=input_ids.device,
        )

        head_mask = self.electra.get_head_mask(
            None, self.electra.config.num_hidden_layers
        )

        # 2) encoder 출력 (LoRA가 여기에 이미 적용됨)
        encoder_outputs = self.electra.encoder(
            embedding_output,
            attention_mask=extended_mask,
            head_mask=head_mask,
            output_attentions=self.electra.config.output_attentions,
            output_hidden_states=self.electra.config.output_hidden_states,
            return_dict=True,
        )
        sequence_output = encoder_outputs.last_hidden_state  # (B, L, hidden_size)

        # 3) add
        fusion = sequence_output + embedding_output
        fusion = self.dropout(fusion)

        # 4) predict embed
        pred_embed = self.to_31(fusion)

        loss = None
        if labels is not None:
            # labels: (B, L), ignore_index=-100
            # target embedding: char_embed[labels]
            with torch.no_grad():
                labels_clamped = labels.clone()
                labels_clamped = labels_clamped.masked_fill(
                    labels_clamped == self.ignore_index, 0
                )
                target_embed = self.char_embed[labels_clamped]  # (B, L, 31)

            # MSE per-token
            mse = (pred_embed - target_embed) ** 2              # (B, L, 31)
            mask = (labels != self.ignore_index).unsqueeze(-1)  # (B, L, 1)
            mse = mse * mask

            denom = mask.sum().clamp(min=1)
            loss = mse.sum() / denom

        return {"loss": loss, "logits": pred_embed}

    def state_dict(self, *args, **kwargs):
        # 1) LoRA(PEFT) 파라미터
        peft_sd = self.peft_model.state_dict(*args, **kwargs)

        # 2) 우리 head / extra 파라미터만 추출
        full_sd = super().state_dict(*args, **kwargs)
        head_sd = {
            k: v for k, v in full_sd.items()
            if k.startswith("to_31")  # 현재 head 이름에 맞게 수정
            # + 필요하면 char_embed도 포함하고 싶을 때:
            # or k.startswith("char_embed")
        }

        # 3) prefix 붙여서 합치기
        peft_sd.update({f"char_head.{k}" for k, v in head_sd.items()})
        return peft_sd

    def load_state_dict(self, state_dict, strict=True):
        # state_dict에서 char_head256.*은 우리 head, 나머지는 peft_model
        my_state = {}
        peft_state = {}
        for k, v in state_dict.items():
            if k.startswith("char_head256."):
                my_state[k.replace("char_head256.", "")] = v
            else:
                peft_state[k] = v

        # LoRA 쪽 로드
        self.peft_model.load_state_dict(peft_state, strict=False)
        # head 로드
        super().load_state_dict(my_state, strict=False)

concat_model = ElectraConcatEmbeddingClassifier(model, num_labels=num_labels)

학습 진행

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


In [ ]:
import evaluate
import numpy as np

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # ignore_index = -100 제거 후 accuracy 계산
    mask = labels != -100
    y_true = labels[mask]
    y_pred = predictions[mask]

    if len(y_true) == 0:
        return {"accuracy": 0.0}

    result = accuracy_metric.compute(predictions=y_pred, references=y_true)
    return {"accuracy": result["accuracy"]}


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="kocharelectra-pron-lora",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    logging_steps=100,
    fp16=True,          # GPU가 지원하면 속도↑
    report_to="none",   # wandb 등 안 쓸 거면 none
    eval_accumulation_steps=16,
    # prediction_loss_only=True,
)

trainer = Trainer(
    model=concat_model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()


In [ ]:
save_dir = "kocharelectra-pron-lora-adapter"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# 발음 vocab도 같이 저장
import json, os
with open(os.path.join(save_dir, "pron_vocab.json"), "w", encoding="utf-8") as f:
    json.dump(pron2id, f, ensure_ascii=False, indent=2)
